In [1]:
# Cell 1: Imports & Environment Setup

import os
from dotenv import load_dotenv

# New imports for Google Generative AI
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS # Still using FAISS for in-memory vector store

# Load environment variables from .env file
load_dotenv()

# Get your Google API key from environment variables
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("Error: GOOGLE_API_KEY environment variable not set.")
    print("Please set your Google API key in a .env file or as an environment variable.")
    # In a notebook, you might want to raise an error to stop execution if the key is critical.
    # raise ValueError("GOOGLE_API_KEY not found.")
else:
    print("Google API key loaded successfully.")

c:\Users\HP\Documents\Project\Contextual_QA_Bot\Contextual-QA-Bot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Google API key loaded successfully.


In [2]:
# Cell 2: Prepare Sample Data

data_file_path = "data.txt"

# Create a dummy data.txt if it doesn't exist for initial testing
if not os.path.exists(data_file_path):
    with open(data_file_path, "w") as f:
        f.write("""LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It provides a modular and flexible architecture that allows developers to chain together different components, such as LLM wrappers, prompt templates, document loaders, and vector stores. This enables building complex applications like chatbots, summarization tools, and question-answering systems.

One of LangChain's core concepts is Retrieval-Augmented Generation (RAG). RAG systems enhance the capabilities of LLMs by allowing them to retrieve information from an external knowledge base before generating a response. This helps overcome the LLM's inherent limitations, such as outdated knowledge or inability to access specific private data. The process typically involves embedding documents into a vector space, storing them in a vector database, and then retrieving relevant documents based on a user's query. The retrieved documents are then provided as context to the LLM, which uses this context to formulate a more accurate and informed answer.

LangChain supports various LLM providers, including OpenAI, Hugging Face, Google, and others. It also integrates with numerous data sources and vector databases. The framework aims to abstract away much of the complexity involved in working directly with LLMs and their ecosystem, making it easier for developers to build powerful AI applications.
""")
    print(f"'{data_file_path}' created with sample content.")
else:
    print(f"'{data_file_path}' already exists.")

'data.txt' already exists.


In [3]:
# Cell 3: Text Loading & Chunking

print("\nLoading and chunking text...")
loader = TextLoader(data_file_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

print(f"Original document split into {len(chunks)} chunks.")
# Optional: Print the first chunk to inspect
# print("\nFirst chunk:")
# print(chunks[0].page_content)


Loading and chunking text...
Original document split into 3 chunks.


In [4]:
# Cell 4: Generate Embeddings & Create Vector Store

print("\nGenerating embeddings and creating vector store (FAISS)...")
# Initialize Google Generative AI embeddings model
# "models/embedding-001" is Google's text embedding model.
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)

# Create an in-memory FAISS vector store from the document chunks and embeddings
# This step sends your text chunks to Google's embedding API.
db = FAISS.from_documents(chunks, embeddings_model)

print("Vector store created successfully.")
print("--- Phase 1 Complete ---")


Generating embeddings and creating vector store (FAISS)...
Vector store created successfully.
--- Phase 1 Complete ---


In [5]:
# New Cell (run this before Cell 5)
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("Error: GOOGLE_API_KEY environment variable not set.")
    print("Please set your Google API key.")
else:
    genai.configure(api_key=google_api_key)
    print("Configured Google Generative AI client.")

    print("\n--- Available Gemini Models ---")
    try:
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                print(f"Name: {m.name}, Description: {m.description}")
            # Optionally, also check for embedding models if you suspect issues there
            # if 'embedContent' in m.supported_generation_methods:
            #     print(f"Name (Embeddings): {m.name}, Description: {m.description}")

    except Exception as e:
        print(f"Error listing models: {e}")
        print("Please check your GOOGLE_API_KEY and ensure Generative Language API is enabled for your project.")

    print("-----------------------------")

Configured Google Generative AI client.

--- Available Gemini Models ---
Name: models/gemini-1.0-pro-vision-latest, Description: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
Name: models/gemini-pro-vision, Description: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
Name: models/gemini-1.5-pro-latest, Description: Alias that points to the most recent production (non-experimental) release of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens.
Name: models/gemini-1.5-pro-001, Description: Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens, released in May of 2024.
Name: models/gemini-1.5-pro-002, Description: Stable version of Gemini 1.5 Pro, 

In [6]:
# Cell 5: Initialize the LLM (Large Language Model)

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

print("\nInitializing Google Gemini LLM...")

# Initialize the ChatGoogleGenerativeAI model
# "gemini-pro" is a text-only model suitable for chat and Q&A.
# temperature controls randomness: 0 for more deterministic, higher for more creative.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0, google_api_key=google_api_key)

print("Gemini LLM initialized successfully.")


Initializing Google Gemini LLM...
Gemini LLM initialized successfully.


In [7]:
# Cell 6: Define the RAG Prompt Template

print("\nDefining the RAG prompt template...")

# This template instructs the LLM to use the provided context to answer the question.
# {context} will be filled by the retrieved documents.
# {input} will be the user's question.
prompt = ChatPromptTemplate.from_template("""Answer the user's question based on the below context.
If the answer is not in the context, clearly state that you don't have enough information from the provided context.
Do not make up information.

Context:
{context}

Question:
{input}
""")

print("Prompt template defined.")


Defining the RAG prompt template...
Prompt template defined.


In [8]:
# Cell 7: Create the Document Combining Chain

print("\nCreating the document combining chain...")

# This chain will take the retrieved documents and the user's question,
# combine them according to the prompt template, and send to the LLM.
document_chain = create_stuff_documents_chain(llm, prompt)

print("Document combining chain created.")


Creating the document combining chain...
Document combining chain created.


In [9]:
# Cell 8: Create the Retrieval Chain

print("\nCreating the retrieval chain...")

# Get the retriever from our FAISS vector store (db from Phase 1)
retriever = db.as_retriever()

# Create the full retrieval chain:
# 1. User input comes in.
# 2. Retriever finds relevant documents from the vector store.
# 3. Document chain combines these documents with the prompt and sends to LLM.
retrieval_chain = create_retrieval_chain(retriever, document_chain)

print("Retrieval chain created.")
print("--- Phase 2 Complete ---")


Creating the retrieval chain...
Retrieval chain created.
--- Phase 2 Complete ---


In [ ]:
# New Cell (e.g., after Cell 8 in main.ipynb)

# --- Core RAG Setup Function ---
# This function encapsulates the setup of your RAG chain.
# We'll call this from our API file later.

def setup_rag_chain():
    """
    Initializes and returns the LangChain RAG retrieval chain
    using Google Generative AI embeddings and LLM.
    """
    # Ensure environment variables are loaded if this function is called independently
    load_dotenv()
    google_api_key = os.getenv("GOOGLE_API_KEY")

    if not google_api_key:
        raise ValueError("GOOGLE_API_KEY environment variable not set.")

    # --- Re-initialize components (similar to Cells 3, 4, 5, 6, 7, 8) ---
    # 1. Text Loading & Chunking
    data_file_path = "data.txt"
    loader = TextLoader(data_file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)

    # 2. Generate Embeddings & Create Vector Store
    embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)
    db = FAISS.from_documents(chunks, embeddings_model)
    retriever = db.as_retriever()

    # 3. Initialize LLM
    llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0, google_api_key=google_api_key)

    # 4. Define the RAG Prompt Template
    prompt = ChatPromptTemplate.from_template("""Answer the user's question based on the below context.
    If the answer is not in the context, clearly state that you don't have enough information from the provided context.
    Do not make up information.

    Context:
    {context}

    Question:
    {input}
    """)

    # 5. Create the Document Combining Chain
    document_chain = create_stuff_documents_chain(llm, prompt)

    # 6. Create the Retrieval Chain
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    return retrieval_chain

# --- Test the function (Optional, run if you want to test within notebook) ---
# print("\nTesting setup_rag_chain function locally...")
# try:
#     my_rag_chain = setup_rag_chain()
#     test_response = my_rag_chain.invoke({"input": "What is LangChain?"})
#     print(f"Test Answer: {test_response['answer']}")
# except Exception as e:
#     print(f"Error during local test: {e}")

In [ ]:
# Cell 9: Implement Q&A Functionality

print("\nTesting the Q&A system. Ask a question about LangChain or RAG based on the provided text.")
print("Type 'exit' to quit.")

while True:
    user_query = input("\nYour Question: ")
    if user_query.lower() == 'exit':
        print("Exiting Q&A system. Goodbye!")
        break

    print("Thinking...")
    # Invoke the retrieval chain with the user's query
    # The 'answer' key in the result will contain the LLM's response.
    try:
        response = retrieval_chain.invoke({"input": user_query})
        print(f"Answer: {response['answer']}")
        # Optional: See the retrieved documents that the LLM used
        # print("\n--- Retrieved Documents (Context for LLM) ---")
        # for doc in response['context']:
        #     print(f"- {doc.page_content[:200]}...") # Print first 200 chars of each doc
        # print("-----------------------------------------------")
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please check your API key, internet connection, and try again.")


Testing the Q&A system. Ask a question about LangChain or RAG based on the provided text.
Type 'exit' to quit.
Thinking...
Answer: Please provide the question you would like me to answer based on the provided context.
Thinking...
Answer: I don't have enough information from the provided context to answer who created LangChain.
Thinking...
Answer: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs).  It provides a modular and flexible architecture allowing developers to chain together different components to build complex applications like chatbots, summarization tools, and question-answering systems.  It supports various LLM providers and integrates with numerous data sources and vector databases.
Thinking...
Answer: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs).  It provides a modular and flexible architecture allowing developers to chain together different compon